# 층 정규화를 수행하는 사용자 정의 층을 구현
RNN을 이용한 자연어 처리에서 종종 쓰인다고 함

In [1]:
import tensorflow as tf
from tensorflow import keras

In [20]:
class LayerNormalization(keras.layers.Layer):
    def __init__(self, eps=0.001, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        
    def build(self, batch_input_shape):
        self.alpha = self.add_weight(name="alpha", dtype=tf.float32,
                                    shape=batch_input_shape[-1: ],
                                    initializer="ones")
        self.beta = self.add_weight(name="beta", dtype=tf.float32,
                                    shape=batch_input_shape[-1: ],
                                    initializer="zeros")
        super().build(batch_input_shape)
        
    def call(self, X):
        mean, var = tf.nn.moments(X, axes=-1, keepdims=True)
        stddev = tf.sqrt(var + self.eps)
        return self.alpha * (X-mean) / (stddev) + self.beta
    
    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "eps": self.eps}

In [23]:
rand_gen = tf.random.Generator.from_seed(42)
temp_w = rand_gen.normal(shape=[2, 10])
temp_w

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702,  0.07595026, -1.2573844 , -0.23193763,
        -1.8107855 ,  0.09988727, -0.50998646, -0.7535805 , -0.57166284],
       [ 0.1480774 , -0.23362993, -0.3522796 ,  0.40621263, -1.0523509 ,
         1.2054597 ,  1.6874489 , -0.4462975 , -2.3410842 ,  0.99009085]],
      dtype=float32)>

In [24]:
custom_LN = LayerNormalization()
LN = keras.layers.LayerNormalization()

custom_out = custom_LN(temp_w)
LN_out = LN(temp_w)

In [25]:
custom_out

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.30888623,  0.88428074,  1.1348636 , -1.1773659 ,  0.60093355,
        -2.1370578 ,  1.1763744 ,  0.11874951, -0.3036841 ,  0.01179214],
       [ 0.13169907, -0.21048042, -0.31684327,  0.363103  , -0.9444185 ,
         1.0795838 ,  1.5116603 , -0.4011251 , -2.0996964 ,  0.8865175 ]],
      dtype=float32)>

In [26]:
LN_out

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.30888614,  0.88428086,  1.1348637 , -1.1773658 ,  0.6009336 ,
        -2.1370575 ,  1.1763746 ,  0.11874961, -0.303684  ,  0.01179225],
       [ 0.13169907, -0.2104804 , -0.31684327,  0.363103  , -0.94441843,
         1.0795836 ,  1.5116602 , -0.4011251 , -2.0996964 ,  0.88651747]],
      dtype=float32)>